# Imports

In [318]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Loading Index Files

In [411]:
# df_eng_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_Mar262021\ESA_website_ENG_old_projects.csv", encoding='utf-8') # dtype fine
df_eng_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads\ESA_website_ENG_new_projects_fixed_encoding_errors.csv", encoding='utf-8') # dtypes fine

# df_fra_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_May062021_fra\ESA_website_FRA_old_projects.csv", encoding='latin1') # dtype fine
# df_fra_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads_fra\ESA_website_FRA_new_projects.csv", encoding='utf-8') # dtypes fine

df_eng_all_projects = pd.read_csv("G:\ESA_downloads\ESA_bundled_with_VCs\ESA_website_ENG_fixed_encoding_errors.csv", encoding='utf-8') # dtypes fine

In [412]:
df_eng_old_projects.Title.iloc[70:80]

70                                                                                                Figure 13D.B-13 Pipe Damage Contingency Logic
71                                                                                                         Figure 1-1 Keystone Pipeline Project
72                                                                      Figure 3-1 Alternative I and II Route and Associated Option for Alberta
73                                                                                  Figure 3-2 Alternative I and II Routing Options in Manitoba
74                                                                                     Figure 3-3 Preferred Alberta–Saskatchewan Pipeline Route
75                                                                                             Figure 3-4 Preferred Manitoba New Pipeline Route
76                        Figure 5-1 Location of the Air Quality Modelling Domain Relative to the Proposed Pipeline Operational Tank Fac

In [413]:
df_eng_old_tables = df_eng_old_projects[df_eng_old_projects['Content Type'] == 'Table']

df_fra_old_tables = df_fra_old_projects[df_fra_old_projects['Type de contenu'] == 'Tableau']

In [414]:
print(len(df_eng_old_projects))
print(len(df_eng_new_projects))
print(len(df_fra_old_projects))
print(len(df_fra_new_projects))
print(len(df_eng_all_projects))
print(len(df_eng_old_projects) + len(df_eng_new_projects))
print(len(df_eng_old_tables))
print(len(df_fra_old_tables))

24285
644
24285
644
24929
24929
15151
15151


# Checking for duplicate rows

In [415]:
left_cols = ['Title', 'Document Number', 'PDF Page Number', 'Hearing order', 'File Name', 'Content Type']
len(df_eng_old_tables[left_cols].drop_duplicates(keep='first'))

15151

In [416]:
df_eng_old_tables[left_cols][df_eng_old_tables[left_cols].duplicated(keep='first')]

,Title,Document Number,PDF Page Number,Hearing order,File Name,Content Type


# Updating old and new index files with index file containing all projects and VCs

In [429]:
df_eng_all_projects.Title = df_eng_all_projects.Title.astype(str)
df_eng_new_projects.Title = df_eng_new_projects.Title.astype(str)
df_eng_old_projects['Regulatory Instrument(s)'] = df_eng_old_projects['Regulatory Instrument(s)'].astype(str)

In [432]:
df_eng_new_projects[df_eng_new_projects.Title.isnull()]

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,ID,Project Download Path,Table Download Path,Good Quality,Data ID,filename,Hearing Order


In [418]:
def merge_dfs_with_matching_specific_columns(left_df, right_df, left_on, right_on):
    """Merge two dataframes based on common columns and only keep rows where
    the values match.
    """
    merged_df = pd.merge(left_df, right_df, how='left', left_on=left_on, right_on=right_on, suffixes=('_x', ''))
    merged_df.drop(merged_df.filter(regex='_x$').columns, axis=1, inplace=True)
    return merged_df

left_cols = ['Title', 'Document Number', 'PDF Page Number']
right_cols = ['Title', 'Document Number', 'PDF Page Number']

df_eng_old_merged = merge_dfs_with_matching_specific_columns(df_eng_old_tables, df_eng_all_projects, left_on=left_cols, right_on=right_cols)
df_eng_new_merged = merge_dfs_with_matching_specific_columns(df_eng_new_projects, df_eng_all_projects, left_on=left_cols, right_on=right_cols)

In [419]:
len(df_eng_old_merged.drop_duplicates(subset=['Title', 'Document Number'])) # Checking to make sure number of rows is the same as the original

15151

In [420]:
# We had encoding issues which led to NaN values in the merged dataframe.
# The errors came from the table titles so I had to manually fix them and make the same in both dataframes.
# For future use, we will need to modify the code to handle encoding issues automatically.
# To do this, we will grab the index file from the database and use that file to label the VCs, then bundle the rows, 
# create French index by merging on ID column, and then concatenate the two dataframes. In other words, concatenate all eng projects in one file and fra in another.

assert len(df_eng_old_merged[df_eng_old_merged.ID.isna()]) == 0 # making sure the merge was successful and did not cause NaN values

In [426]:
df_eng_old_tables.dtypes

Title                          object
Content Type                   object
Application Name               object
Application Short Name         object
Application Filing Date        object
Company Name                   object
Commodity                      object
File Name                      object
ESA Folder URL                 object
Document Number                object
PDF Download URL               object
Application Type (NEB Act)     object
Pipeline Location              object
Hearing order                  object
Consultant Name                object
Pipeline Status                object
Regulatory Instrument(s)       object
Application URL                object
Decision URL                   object
ESA Section(s)                 object
ESA Section(s) Index          float64
ESA Section(s) Topics          object
PDF Page Number                 int64
PDF Page Count                float64
PDF Size                      float64
PDF Outline                    object
ID          

In [430]:
df_eng_new_projects.dtypes

Title                          object
Content Type                   object
Application Name               object
Application Short Name         object
Application Filing Date        object
Company Name                   object
Commodity                      object
File Name                      object
ESA Folder URL                 object
Document Number                object
PDF Download URL               object
Application Type (NEB Act)     object
Pipeline Location              object
Hearing order                  object
Consultant Name                object
Pipeline Status                object
Regulatory Instrument(s)      float64
Application URL                object
Decision URL                   object
ESA Section(s)                 object
ESA Section(s) Index          float64
ESA Section(s) Topics         float64
PDF Page Number                 int64
PDF Page Count                  int64
PDF Size                      float64
PDF Outline                   float64
ID          

In [421]:
df_eng_new_merged[df_eng_new_merged.ID.isna()] # making sure the merge was successful and did not cause NaN values

,Title,Document Number,PDF Page Number,filename,Hearing Order,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Count,PDF Size,PDF Outline,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights,ID,Project Download Path,Table Download Path,Good Quality
182,Table 2‐1 Extent of Land Use and Soil Units within the PDA,A7I8X3,41,wstpthdlvr2022_table-21-extent-of-land-use-and-soil-units-within-_pt-1_pg-41_doc-num-A7I8X3.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,Table 3‐1 Agricultural Suitability Classes,A7I8X3,44,wstpthdlvr2022_table-31-agricultural-suitability-classes_pt-1_pg-44_doc-num-A7I8X3.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,Table 3‐2 Agricultural Suitability Subclasses,A7I8X3,45,wstpthdlvr2022_table-32-agricultural-suitability-subclasses_pt-1_pg-45_doc-num-A7I8X3.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,Table 3‐3 Relative Susceptibility of Soil to Wind Erosion Based on Soil Texture,A7I8X3,46,wstpthdlvr2022_table-33-relative-susceptibility-of-soil-to-wind-e_pt-1_pg-46_doc-num-A7I8X3.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,Table 3‐4 Relative Susceptibility to Water Erosion Based on Soil Texture,A7I8X3,47,wstpthdlvr2022_table-34-relative-susceptibility-to-water-erosion-_pt-1_pg-47_doc-num-A7I8X3.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,Table 3‐5 Slope Length (L) and Gradient (S) Values Associated with Various Landforms of Alberta,A7I8X3,48,wstpthdlvr2022_table-35-slope-length-l-and-gradient-s-values-asso_pt-1_pg-48_doc-num-A7I8X3.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,Table 3‐6 Compaction Risk Rating Matrix,A7I8X3,49,wstpthdlvr2022_table-36-compaction-risk-rating-matrix_pt-1_pg-49_doc-num-A7I8X3.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,Table 4‐1 Caroline Soil Unit Summary,A7I8X3,51,wstpthdlvr2022_table-41-caroline-soil-unit-summary_pt-1_pg-51_doc-num-A7I8X3.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Creating the Merged French Index

In [344]:
df_fra_old_tables.head()

,Titre,Type de contenu,Nom de la demande,Nom abrégé de la demande,Dépôt de la demande,Nom de la société,Produit de base,Nom de fichier,URL du dossier de l'ÉES,Numéro de document,URL de téléchargement PDF,Type de demande (Loi sur l'Office national de l'énergie),Emplacement du pipeline,Ordonnance d'audience,Nom du consultant,État d'avancement,Instruments réglementaires,URL de la demande,URL de la décision,Sections de l'EES,Index des sections de l'ÉES,Sujets des sections de l'ÉES,Numéro de page PDF,Nombre de pages PDF,Taille PDF,Aperçu PDF,ID,Chemin d'accès pour télécharger le projet,Chemin d'accès pour télécharger le tableau,Bonne qualité
9134,TABLEAU 1 EMPLACEMENTS DES RELEVÉS DE LA VÉGÉTATION EN 2012,Tableau,Demande visant le projet pipelinier d'Edmonton à Hardisty,Edmonton à Hardisty,12/14/2012,Enbridge Pipelines Inc.,Pétrole,B2-31 - Volume IIA - ESA - Appendix 8 - Vegetation Survey - A3E3A6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/895002,A3E3A6,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/894928,Projets de grande envergure (plus de 40 km),Alberta,OH-001-2013,Stantec Consulting Ltd,En exploitation,OC-062,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/895427,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%A9ment/Afficher/2411664,Annexe 8 : Inventaire de la végétation,32.0,Végétation,9,106.0,5.98,Oui,0,/projects/dmntnhrdst.zip,/tables/dmntnhrdst_tableau-1-emplacements-des-relevés-de-la-végétatio_pt-1_pg-9_num-du-doc-A3E3A6.zip,True
9135,TABLEAU 1 COMMUNAUTÉS AUTOCHTONES ET DISTANCE DE L'EMPRISE DU PIPELINE PROPOSÉ,Tableau,Demande relative au projet de Horn River,Horn River,2/19/2010,NOVA Gas Transmission Ltd.,Gaz,B-02-31 - NGTL Horn River Project - ESA Appendix 6 TLU Report Part 1 of 11 - A1R8F8,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/600712,A1R8F8,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/600731,Projets de grande envergure (plus de 40 km),Colombie-Britannique,GH-2-2010,TERA Environmental Consultants,En exploitation,GC-117,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/601085,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%A9ment/Afficher/665334,Annexe 6 : Étude exhaustive sur l'utilisation des terres à des fins traditionnelles pour le projet de la canalisation principale Horn River de NOVA Gas Transmission Ltd.,31.0,"Terres, Air, Eau, Aspect humain, Utilisation des terres à des fins traditionnelles",9,57.0,2.33,Oui,1,/projects/hrnrvr.zip,/tables/hrnrvr_tableau-1-communautés-autochtones-et-distance-de-l_pt-1_pg-9_num-du-doc-A1R8F8.zip,True
9136,TABLEAU 1 ZONES ÉVALUÉES EN VERTU DU PERMIS 07-368,Tableau,Demande relative au projet pipelinier Redwillow,Redwillow,12/7/2007,SemCAMS Redwillow ULC,Gaz,B-2ff - Appendix X - Part 1 - A1C3Y6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/487605,A1C3Y6,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/487965,Projets de grande envergure (plus de 40 km),"Alberta, Colombie-Britannique",GH-2-2008,TERA Environmental Consultants,Certificats révoqués,"GC-114, RO-GC-114",https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/487906,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%A9ment/Afficher/551651,"Annexe X : Évaluation des répercussions sur les ressources historiques, Évaluation des répercussions sur les ressources archéologiques et paléontologie",32.0,Aspect humain,7,30.0,1.52,Non,2,/projects/rdwllw.zip,/tables/rdwllw_tableau-1-zones-évaluées-en-vertu-du-permis-07-368_pt-1_pg-7_num-du-doc-A1C3Y6.zip,True
9137,TABLEAU 1 ZONES ÉVALUÉES EN VERTU DU PERMIS 2007-252,Tableau,Demande relative au projet pipelinier Redwillow,Redwillow,12/7/2007,SemCAMS Redwillow ULC,Gaz,B-2gg - Appendix X - Part 2 - A1C3Y7,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/487605,A1C3Y7,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/487871,Projets de grande envergure (plus de 40 km),"Alberta, Colombie-Britannique",GH-2-2008,TERA Environmental Consultants,Certificats révoqués,"GC-114, RO-GC-114",https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/487906,https://apps.cer-rec.gc

In [348]:
right_cols = ['ID']
left_cols = ['ID']

In [144]:
# Checking to see that the columns we've chosen to identify the rows are the same for each item in the dataframes

mismatched_fr_items = []
mismatched_en_items = []

for right_col, left_col in zip(right_cols, left_cols):
    fr_test = df_fra_old_tables[left_col].unique()
    en_test = df_eng_old_tables[right_col].unique()
    
    for fr_item, en_item in zip(fr_test, en_test):
        if fr_item != en_item:
            print(fr_item, en_item)
            mismatched_fr_items.append(fr_item)
            mismatched_en_items.append(en_item)

print(len(mismatched_fr_items))

0


In [112]:
df_fra_old_merged = merge_dfs_with_matching_specific_columns(df_fra_old_tables, df_eng_all_projects, left_on=left_cols, right_on=right_cols)
df_fra_new_merged = merge_dfs_with_matching_specific_columns(df_fra_new_projects, df_eng_all_projects, left_on=left_cols, right_on=right_cols)

In [349]:
df_fra_old_merged = merge_dfs_with_matching_specific_columns(df_eng_old_merged, df_fra_old_projects, left_on=left_cols, right_on=right_cols)
df_fra_new_merged = merge_dfs_with_matching_specific_columns(df_eng_new_merged, df_fra_new_projects, left_on=left_cols, right_on=right_cols)

In [350]:
print(len(df_fra_old_merged))

15151


In [352]:
df_fra_old_merged.head(2)

,Title,File Name,Document Number,Hearing order,PDF Page Number,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,ESA Folder URL,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Count,PDF Size,PDF Outline,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights,ID,Project Download Path,Table Download Path,Good Quality,Titre,Type de contenu,Nom de la demande,Nom abrégé de la demande,Dépôt de la demande,Nom de la société,Produit de base,Nom de fichier,URL du dossier de l'ÉES,Numéro de document,URL de téléchargement PDF,Type de demande (Loi sur l'Office national de l'énergie),Emplacement du pipeline,Ordonnance d'audience,Nom du consultant,État d'avancement,Instruments réglementaires,URL de la demande,URL de la décision,Sections de l'EES,Index des sections de l'ÉES,Sujets des sections de l'ÉES,Numéro de page PDF,Nombre de pages PDF,Taille PDF,Aperçu PDF,Chemin d'accès pour télécharger le projet,Chemin d'accès pour télécharger le tableau,Bonne qualité
0,TABLE 1 2012 VEGETATION SURVEY LOCATIONS,B2-31 - Volume IIA - ESA - Appendix 8 - Vegetation Survey - A3E3A6,A3E3A6,OH-001-2013,9,Table,Application for the Edmonton to Hardisty Pipeline Project,Edmonton to Hardisty,12/14/2012,Enbridge Pipelines Inc.,Oil,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/895002,894928,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/894928,Large Projects (over 40 km),Alberta,Stantec Consulting Ltd,Operating,OC-062,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/895427,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2411664,Appendix 8: Vegetation Survey,32.0,Vegetation,106.0,5.98,Yes,0.0,1.0,18.0,4.0,2.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,1.0,0.0,0.0,5.0,0,/projects/dmntnhrdst.zip,/tables/dmntnhrdst_table-1-2012-vegetation-survey-locations_pt-1_pg-9_doc-num-A3E3A6.zip,True,TABLEAU 1 EMPLACEMENTS DES RELEVÉS DE LA VÉGÉTATION EN 2012,Tableau,Demande visant le projet pipelinier d'Edmonton à Hardisty,Edmonton à Hardisty,12/14/2012,Enbridge Pipelines Inc.,Pétrole,B2-31 - Volume IIA - ESA - Appendix 8 - Vegetation Survey - A3E3A6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/895002,A3E3A6,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/894928,Projets de grande envergure (plus de 40 km),Alberta,OH-001-2013,Stantec Consulting Ltd,En exploitation,OC-062,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/895427,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%A9ment/Afficher/2411664,Annexe 8 : Inventaire de la végétation,32.0,Végétation,9,106.0,5.98,Oui,/projects/dmntnhrdst.zip,/tables/dmntnhrdst_tableau-1-emplacements-des-relevés-de-la-végétatio_pt-1_pg-9_num-du-doc-A3E3A6.zip,True
1,TABLE 1 ABORIGINAL COMMUNITIES AND THEIR DISTANCE FROM THE PROPOSED PIPELINE RIGHT-OF-WAY,B-02-31 - NGTL Horn River Project - ESA Appendix 6 TLU Report Part 1 of 11 - A1R8F8,A1R8F8,GH-2-2010,9,Table,Application for the Horn River Project,Horn River,2/19/2010,NOVA Gas Transmission Ltd.,Gas,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/600712,600731,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/600731,Large Projects (over 40 km),British Columbia,TERA Environmental Consultants,Operating,GC-117,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/601085,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/665334,Appendix 6: Comprehensive Traditional Land Use St

In [353]:
df_fra_new_merged.head()

,Title,File Name,Document Number,Hearing order,PDF Page Number,filename,Hearing Order,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,ESA Folder URL,PDF Download URL,Application Type (NEB Act),Pipeline Location,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Count,PDF Size,PDF Outline,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights,ID,Project Download Path,Table Download Path,Good Quality,Titre,Type de contenu,Nom de la demande,Nom abrégé de la demande,Dépôt de la demande,Nom de la société,Produit de base,Nom de fichier,URL du dossier de l'ÉES,Numéro de document,URL de téléchargement PDF,Type de demande (Loi sur l'Office national de l'énergie),Emplacement du pipeline,Ordonnance d'audience,Nom du consultant,État d'avancement,Instruments réglementaires,URL de la demande,URL de la décision,Sections de l'EES,Index des sections de l'ÉES,Sujets des sections de l'ÉES,Numéro de page PDF,Nombre de pages PDF,Taille PDF,Aperçu PDF,Chemin d'accès pour télécharger le projet,Chemin d'accès pour télécharger le tableau,Bonne qualité,Data ID
0,TABLE 1 TIMEFRAMES FOR SUBMITTING COPY #1 OF THE MANIFEST DOCUMENT AND MANIFEST RECONCILIATION,C08618-3 Attachment CER 1.4-1_Part 1 of 6_ESA_Appendix A - A7I8X1,A7I8X1,NaN,204,wstpthdlvr2022_table-1-timeframes-for-submitting-copy-1-of-the-ma_pt-1_pg-204_doc-num-A7I8X1.csv,NaN,Table,Application for the NGTL West Path Delivery 2022 Project,West Path Delivery 2022,6/1/2020,NOVA Gas Transmission Ltd.,Gas,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3970826,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/3969837,CERA 183 Pipeline Facilities Certificate,Alberta,Stantec Consulting Ltd,Approved,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3930580,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4091795,APPENDIX A Environmental Protection Plans,NaN,NaN,236.0,12.46,Yes,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,76.0,/projects/wstpthdlvr2022.zip,/tables/wstpthdlvr2022_table-1-timeframes-for-submitting-copy-1-of-the-ma_pt-1_pg-204_doc-num-A7I8X1.zip,True,"Tableau 13-12 Niveau d'instruction, 2019",Tableau,Demande visant le projet de livraison parcours ouest de NGTL en 2022,Livraison parcours ouest 2022,6/1/2020,NOVA Gas Transmission Ltd.,Gaz,C08618-8 Attachment CER 1.4-1_Part 6 of 6_ESA_Section 7-18 - A7I8X6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/3970826,A7I8X6,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/3970830,CERA 183 certificat d'établissement de pipeline,Alberta,NaN,Stantec Consulting Ltd,Demande approuvée,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/3930580,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4091795,"7.0 ÉVALUATION DES EFFETS SUR LA FAUNE ET L'HABITAT FAUNIQUE, 8.0 ÉVALUATION DES EFFETS SUR LES RESSOURCES AQUATIQUES, 9.0 ÉVALUATION DES ÉMISSIONS DE GAZ À EFFET DE SERRE, 10.0 ÉVALUATION DES EFFETS SUR L'OCCUPATION HUMAINE ET L'UTILISATION DES RESSOURCES, 11.0 ÉVALUATION DES EFFETS SUR L'UTILISATION DES TERRES ET DES RESSOURCES À DES FINS TRADITIONNELLES, 12.0 ÉVALUATION DES EFFETS SUR LES RESSOURCES PATRIMONIALES, 13.0 ÉVALUATION DES EFFETS SUR LES COMPOSANTES SOCIOÉCONOMIQUES VALORISÉES, 14.0 DROITS DES PEUPLES AUTOCHTONES, 15.0 ÉVALUATION DES EFFETS DE L'ENVIRONNEMENT SUR LE PROJET, 16.0 ACCIDENTS ET DÉFAILLANCES 17.0 CONCLUSIONS, 18.0 RÉFÉRENCES",NaN,NaN

In [354]:
df_fra_old_merged.to_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads_fra\df_fra_old_merged.csv", index=False, encoding='utf-8')